In [ ]:
from math import sqrt
from random import randint,shuffle
import random
import time

# Defining infinity
INF = 1e18
mult = 1e6
 
# Structure to represent a 2D point
class Point :
    def __init__(self,X=0,Y=0) -> None:
        self.X=X
        self.Y=Y
        
# Structure to represent a 2D circle
class Circle :
    def __init__(self,c=Point(),r=0) -> None:       
        self.C=c
        self.R=r
 
 
# Function to return the euclidean distance
# between two points
def dist(a, b):
    return sqrt(pow(a.X - b.X, 2)
                + pow(a.Y - b.Y, 2))

# Function to check whether a point lies inside
# or on the boundaries of the circle
def is_inside(c, p):
    return dist(c.C, p) <= c.R
 
# The following two functions are used
# To find the equation of the circle when
# three points are given.
 
# Helper method to get a circle defined by 3 points
def get_circle_center(bx, by,
                        cx, cy):
    B = bx * bx + by * by
    C = cx * cx + cy * cy
    D = bx * cy - by * cx
    return Point((cy * B - by * C) / (2 * D),
             (bx * C - cx * B) / (2 * D))
 
# Function to return the smallest circle
# that intersects 2 points
def circle_from1(A, B):
    # Set the center to be the midpoint of A and B
    C = Point((A.X + B.X) / 2.0, (A.Y + B.Y) / 2.0 )
 
    # Set the radius to be half the distance AB
    return Circle(C, dist(A, B) / 2.0 )
 
# Function to return a unique circle that
# intersects three points
def circle_from2(A, B, C):
    I = get_circle_center(B.X - A.X, B.Y - A.Y,
                                C.X - A.X, C.Y - A.Y)
 
    I.X += A.X
    I.Y += A.Y
    return Circle(I, dist(I, A))
 
 
 
 
 
# Function to check whether a circle
# encloses the given points
def is_valid_circle(c, P):
 
    # Iterating through all the points
    # to check  whether the points
    # lie inside the circle or not
    for p in P:
        if (not is_inside(c, p)):
            return False
    return True
 
# Function to return the minimum enclosing
# circle for N <= 3
def min_circle_trivial(P):
    assert(len(P) <= 3)
    if not P :
        return Circle()
     
    elif (len(P) == 1) :
        return Circle(P[0], 0)
     
    elif (len(P) == 2) :
        return circle_from1(P[0], P[1])
     
 
    # To check if MEC can be determined
    # by 2 points only
    for i in range(3):
        for j in range(i + 1,3):
 
            c = circle_from1(P[i], P[j])
            if (is_valid_circle(c, P)):
                return c
         
     
    return circle_from2(P[0], P[1], P[2])
 
# Returns the MEC using Welzl's algorithm
# Takes a set of input points P and a set R
# points on the circle boundary.
# n represents the number of points in P
# that are not yet processed.
def welzl_helper(P, R, n):
    # Base case when all points processed or |R| = 3
    if (n == 0 or len(R) == 3) :
        return min_circle_trivial(R)
     
 
    # Pick a random point randomly
    idx = randint(0,n-1)
    p = P[idx]
 
    # Put the picked point at the end of P
    # since it's more efficient than
    # deleting from the middle of the vector
    P[idx], P[n - 1]=P[n-1],P[idx]
 
    # Get the MEC circle d from the
    # set of points P - :p
    d = welzl_helper(P, R.copy(), n - 1)
 
    # If d contains p, return d
    if (is_inside(d, p)) :
        return d
     
 
    # Otherwise, must be on the boundary of the MEC
    R.append(p)
 
    # Return the MEC for P - :p and R U :p
    return welzl_helper(P, R.copy(), n - 1)

def minimum_enclosing_circle(P):
    P_copy = P.copy()
    shuffle(P_copy)
    return welzl_helper(P_copy, [], len(P_copy))

if __name__=='__main__':
    alt = 1000                     
    lat_position = 24.788090       
    lon_position = 120.997858      
    TxR = 300
    lat = [0]*101
    lon = [0]*101
    laspeed = [0]*101
    lospeed = [0]*101
    lospeed_total = 0
    laspeed_total = 0
    latmin = 24.7890
    latmax = 24.7895
    lonmin = 120.9975
    lonmax = 120.9980
    smax = 200*0.000009
    smin = 20*0.000009
    user = 100
    count_out = 0
    count_num = 0
    time_max = 0
    ladiff_total = 0
    lodiff_total = 0
    count_lad = 0
    count_lod = 0
    
    
    
    f= open("users_6G.txt","r")
    while True:
        laspeed_total = 0
        lospeed_total = 0
        count_num += 1
        start = time.time()
        print("Current users number = ",user)
        for i in range(user):
            line = f.readline()
            lat[i] = float(line)
            line = f.readline()
            lon[i] = float(line)
            line = f.readline()
            laspeed[i] = float(line)
            line = f.readline()
            lospeed[i] = float(line)
            
            
        v1 = [Point(lon[0]*mult,lat[0]*mult)]
        for i in range(1, user):
            v1.append(Point(lon[i]*mult,lat[i]*mult))
        mec = minimum_enclosing_circle(v1)
        
        for i in range(0,user):
            lospeed_total += lospeed[i];
            laspeed_total += laspeed[i];
        lospeed_uav = lospeed_total/user;
        laspeed_uav = laspeed_total/user;

        
        if mec.R/mult > TxR*0.000009:
            print("Out of range")
            for i in range(user-1 , -1, -1): # 99~0
                v2 = [Point(lon[user-1]*mult,lat[user-1]*mult)]
                for j in range(user-2 , i-1, -1): #user-i
                    v2.append(Point(lon[j]*mult,lat[j]*mult))
                mec2 = minimum_enclosing_circle(v2);
                if mec2.R/mult > TxR*0.000009:
                    print("Kicked off user below ",i)
                    v3 = [Point(lon[user-1]*mult,lat[user-1]*mult)]
                    for j in range(user-2 , i ,-1):
                        v3.append(Point(lon[j]*mult,lat[j]*mult))
                    mec3 = minimum_enclosing_circle(v3)
                    for m in range(i+1):
                        if sqrt(pow(lat[m] - mec3.C.Y/mult, 2) + pow(lon[m] - mec3.C.X/mult, 2)) > TxR * 0.000009:
                            print("Kick off user ",m)
                            lat[m] = 0
                    cnt = 0
                    for k in range(user):
                        if lat[k] != 0:
                            #print("lat[",cnt,"] = lat[",k,"]")
                            lat[cnt] = lat[k]
                            cnt += 1
                    user = cnt
                    if mec3.C.Y/mult > lat_position:
                        print("前移",(mec3.C.Y/mult - lat_position) / 0.000009,"公尺")
                    else:
                        print("後移",(lat_position - mec3.C.Y/mult) / 0.000009,"公尺")
                    if mec3.C.X/mult > lon_position:
                        print("左移",(mec3.C.X/mult - lon_position) / 0.000009,"公尺")
                    else:
                        print("右移",(lon_position - mec3.C.X/mult) / 0.000009,"公尺")
                    lat_position = mec3.C.Y/mult
                    lon_position = mec3.C.X/mult
                    break
        elif mec.R/mult == 0:
            print("mec.Y = %5.6f" %mec.C.Y/mult)
            print("mec.X = %5.6f" %mec.C.X/mult)
            if mec.C.Y/mult > lat_position:
                print("前移",(mec.C.Y/mult - lat_position) / 0.000009,"公尺")
            else:
                print("後移",(lat_position - mec.C.Y/mult) / 0.000009,"公尺")
            if mec.C.X/mult > lon_position:
                print("左移",(mec.C.X/mult - lon_position) / 0.000009,"公尺")
            else:
                print("右移",(lon_position - mec.C.X/mult) / 0.000009,"公尺")
            
            
            lat_position = mec.C.Y/mult
            lon_position = mec.C.X/mult
            print("finish")
           
        else:
            print("lat = ",mec.C.Y/mult)
            print("lon = ",mec.C.X/mult)
            #print("R = ",mec.R/mult)
            if mec.C.Y/mult > lat_position:
                print("前移",(mec.C.Y/mult - lat_position) / 0.000009,"公尺")
            else:
                print("後移",(lat_position - mec.C.Y/mult) / 0.000009,"公尺")
            if mec.C.X/mult > lon_position:
                print("左移",(mec.C.X/mult - lon_position) / 0.000009,"公尺")
            else:
                print("右移",(lon_position - mec.C.X/mult) / 0.000009,"公尺")
            print("real laspeed = ",abs(lat_position - mec.C.Y/mult) / 0.000009/0.03,"m/s")
            print("real lospeed = ",abs(lon_position - mec.C.X/mult) / 0.000009/0.03,"m/s")
            print("predict laspeed = ",laspeed_uav/0.03,"m/s")
            print("predict lospeed = ",lospeed_uav/0.03,"m/s")
            larspd = abs((lat_position - mec.C.Y/mult) / 0.000009/0.03)
            lorspd = abs((lon_position - mec.C.X/mult) / 0.000009/0.03)
            lapspd = abs(laspeed_uav/0.03)
            lopspd = abs(lospeed_uav/0.03)
            if larspd != 0:
                ladiff = abs((larspd - lapspd)/larspd)
            if lorspd != 0:
                lodiff = abs((lorspd - lopspd)/lorspd)
            
            if count_num > 1:
                ladiff_total = ladiff_total + ladiff
                lodiff_total = lodiff_total + lodiff
                ladiff_mean = ladiff_total/(count_num-1)
                lodiff_mean = lodiff_total/(count_num-1)
                if ladiff > 0.2:
                    count_lad += 1
                if lodiff > 0.2:
                    count_lod += 1
                print("la誤差率 = ", ladiff_mean)
                print("lo誤差率 = ", lodiff_mean)
            
            lat_position = mec.C.Y/mult
            lon_position = mec.C.X/mult
            print("finish")
            
        end = time.time()
        t = end - start
        if t < 0.03:
            time.sleep(0.03-t)

Current users number =  100
lat =  24.79175
lon =  120.999764
前移 406.6666666666663 公尺
左移 211.77777777836582 公尺
real laspeed =  13555.555555555544 m/s
real lospeed =  7059.259259278861 m/s
predict laspeed =  20.72853766666666 m/s
predict lospeed =  20.838305666666667 m/s
finish
Current users number =  100
lat =  24.791756
lon =  120.999769
前移 0.666666666562479 公尺
左移 0.5555555557318964 公尺
real laspeed =  22.222222218749298 m/s
real lospeed =  18.518518524396548 m/s
predict laspeed =  20.618079666666663 m/s
predict lospeed =  20.757115000000002 m/s
la誤差率 =  0.07218641485499999
lo誤差率 =  0.12088420964421626
finish
Current users number =  100
lat =  24.7917615
lon =  120.999774
前移 0.6111111111471877 公尺
左移 0.5555555557318964 公尺
real laspeed =  20.370370371572925 m/s
real lospeed =  18.518518524396548 m/s
predict laspeed =  20.70194733333333 m/s
predict lospeed =  20.717851333333336 m/s
la誤差率 =  0.044231914670229466
lo誤差率 =  0.11982409064455275
finish
Current users number =  100
lat =  24.7917

Current users number =  100
lat =  24.7918915
lon =  120.9998905
前移 0.666666666562479 公尺
左移 0.5555555557318964 公尺
real laspeed =  22.222222218749298 m/s
real lospeed =  18.518518524396548 m/s
predict laspeed =  20.628171999999996 m/s
predict lospeed =  20.696017666666666 m/s
la誤差率 =  0.06413322574598097
lo誤差率 =  0.10918388368698405
finish
Current users number =  100
Out of range
Kicked off user below  27
Kick off user  27
前移 12.000000000098352 公尺
右移 7.3333333337662525 公尺
Current users number =  99
Out of range
Kicked off user below  27
Kick off user  27
前移 0.666666666562479 公尺
左移 0.44444444490131396 公尺
Current users number =  98
Out of range
Kicked off user below  28
Kick off user  28
前移 0.7222222223725162 公尺
左移 0.3888888879070388 公尺
Current users number =  97
Out of range
Kicked off user below  30
Kick off user  30
前移 0.7222222223725162 公尺
左移 0.44444444490131396 公尺
Current users number =  96
Out of range
Kicked off user below  33
Kick off user  33
前移 0.6111111111471877 公尺
左移 0.4444444

Current users number =  51
lat =  24.79167850464278
lon =  121.00015760517722
後移 49.44392858005939 公尺
左移 37.067241913114735 公尺
real laspeed =  1648.1309526686464 m/s
real lospeed =  1235.5747304371578 m/s
predict laspeed =  21.478822875817002 m/s
predict lospeed =  21.450779738562098 m/s
la誤差率 =  0.19592452917489786
lo誤差率 =  0.2308163844396414
finish
Current users number =  51
Out of range
Kicked off user below  22
Kick off user  22
前移 50.2217063578472 公尺
右移 36.511686357382835 公尺
Current users number =  50
lat =  24.79168332023185
lon =  121.00016416290931
後移 49.68664090535905 公尺
左移 37.2403232565451 公尺
real laspeed =  1656.2213635119683 m/s
real lospeed =  1241.3441085515035 m/s
predict laspeed =  21.314838666666674 m/s
predict lospeed =  21.698190666666672 m/s
la誤差率 =  0.20455179558913913
lo誤差率 =  0.23836548063405932
finish
Current users number =  50
Out of range
Kicked off user below  21
Kick off user  21
前移 50.46441868314686 公尺
右移 36.684767700813204 公尺
Current users number =  49
lat

Current users number =  30
Out of range
Kicked off user below  9
Kick off user  9
前移 0.05555555541529126 公尺
右移 37.50000000006063 公尺
Current users number =  29
lat =  24.791837786226232
lon =  121.00021717469185
後移 41.9126415296647 公尺
左移 36.5194102064 公尺
real laspeed =  1397.0880509888232 m/s
real lospeed =  1217.313673546667 m/s
predict laspeed =  19.839020689655168 m/s
predict lospeed =  20.88881034482759 m/s
la誤差率 =  0.32116001919269577
lo誤差率 =  0.3367478705298079
finish
Current users number =  29
lat =  24.791710753632124
lon =  121.0002296676052
後移 14.114732678649489 公尺
左移 1.3881014824493023 公尺
real laspeed =  470.4910892883163 m/s
real lospeed =  46.27004941497675 m/s
predict laspeed =  22.160085057471267 m/s
predict lospeed =  21.072140229885054 m/s
la誤差率 =  0.3274774199624876
lo誤差率 =  0.33882622763054115
finish
Current users number =  29
Out of range
Kicked off user below  21
Kick off user  7
Kick off user  21
前移 58.02737420839637 公尺
右移 73.90751168796012 公尺
Current users number 

predict lospeed =  20.566956666666666 m/s
la誤差率 =  0.4183729244473693
lo誤差率 =  0.42170681942308835
finish
Current users number =  20
lat =  24.792262
lon =  121.0002695
前移 30.535776645419016 公尺
左移 9.055544591913328 公尺
real laspeed =  1017.8592215139672 m/s
real lospeed =  301.85148639711093 m/s
predict laspeed =  19.542176666666666 m/s
predict lospeed =  20.47960833333333 m/s
la誤差率 =  0.4226992920112095
lo誤差率 =  0.42563333129400877
finish
Current users number =  20
Out of range
Kicked off user below  7
Kick off user  7
前移 1.0555555552590097 公尺
右移 37.944444444961945 公尺
Current users number =  19
lat =  24.792198358540013
lon =  120.99999498339638
後移 8.126828887276991 公尺
左移 7.442599597753643 公尺
real laspeed =  270.8942962425664 m/s
real lospeed =  248.0866532584548 m/s
predict laspeed =  19.800450877192986 m/s
predict lospeed =  19.168666666666667 m/s
la誤差率 =  0.4233167807358573
lo誤差率 =  0.426174750415567
finish
Current users number =  19
lat =  24.79184913331723
lon =  121.0002630722137

...

前移 14.111111111405863 公尺
左移 58.4444444437319 公尺
real laspeed =  470.3703703801955 m/s
real lospeed =  1948.1481481243968 m/s
predict laspeed =  23.83888666666667 m/s
predict lospeed =  23.843293333333335 m/s
la誤差率 =  0.9423037489283121
lo誤差率 =  0.9487443597145433
finish
Current users number =  5
lat =  24.7930285
lon =  121.001809
後移 7.8333333336881115 公尺
左移 11.388888888556417 公尺
real laspeed =  261.1111111229371 m/s
real lospeed =  379.62962961854726 m/s
predict laspeed =  23.582046666666663 m/s
predict lospeed =  19.29022 m/s
la誤差率 =  0.9422945243892283
lo誤差率 =  0.9487444848214
finish
Current users number =  5
lat =  24.792698
lon =  121.0013715
後移 36.72222222187808 公尺
右移 48.61111110996161 公尺
real laspeed =  1224.0740740626027 m/s
real lospeed =  1620.3703703320539 m/s
predict laspeed =  17.8867 m/s
predict lospeed =  22.998120000000004 m/s
la誤差率 =  0.9423067078901989
lo誤差率 =  0.9487549633030772
finish
Current users number =  5
lat =  24.793369640374994
lon =  121.00098183461695
前移 7

finish
Current users number =  5
lat =  24.793439811782303
lon =  121.00003263188233
前移 91.9235313669257 公尺
右移 197.98534640674208 公尺
real laspeed =  3064.117712230857 m/s
real lospeed =  6599.511546891403 m/s
predict laspeed =  24.750753333333336 m/s
predict lospeed =  17.990900000000003 m/s
la誤差率 =  0.942219941351513
lo誤差率 =  0.9490527947569014
finish
Current users number =  5
lat =  24.793043682801244
lon =  121.00263570032111
後移 44.0143312287786 公尺
左移 289.22982653038315 公尺
real laspeed =  1467.1443742926201 m/s
real lospeed =  9640.99421767944 m/s
predict laspeed =  24.60002 m/s
predict lospeed =  22.565933333333337 m/s
la誤差率 =  0.9422314424112453
lo誤差率 =  0.9490664253183677
finish
Current users number =  5
lat =  24.793526
lon =  121.0012895
前移 53.59079986178017 公尺
右移 149.57781345654237 公尺
real laspeed =  1786.3599953926723 m/s
real lospeed =  4985.927115218079 m/s
predict laspeed =  19.947953333333334 m/s
predict lospeed =  18.096873333333335 m/s
la誤差率 =  0.9422445070991102
lo誤差率 

Current users number =  5
lat =  24.792424
lon =  121.001025
前移 15.99999999986797 公尺
右移 86.05555555618594 公尺
real laspeed =  533.3333333289323 m/s
real lospeed =  2868.518518539531 m/s
predict laspeed =  20.052493333333334 m/s
predict lospeed =  17.890813333333334 m/s
la誤差率 =  0.9423422586665183
lo誤差率 =  0.9493454098998465
finish
Current users number =  5
lat =  24.7929865
lon =  121.001197
前移 62.50000000010105 公尺
左移 19.111111111808693 公尺
real laspeed =  2083.333333336702 m/s
real lospeed =  637.0370370602898 m/s
predict laspeed =  21.870520000000003 m/s
predict lospeed =  21.876953333333336 m/s
la誤差率 =  0.9423553805061771
lo誤差率 =  0.9493499488149916
finish
Current users number =  5
lat =  24.79312
lon =  121.0017275
前移 14.833333332988888 公尺
左移 58.94444444404851 公尺
real laspeed =  494.44444443296294 m/s
real lospeed =  1964.8148148016169 m/s
predict laspeed =  23.83888666666667 m/s
predict lospeed =  23.843293333333335 m/s
la誤差率 =  0.9423580039111398
lo誤差率 =  0.9493606622825937
finish


predict lospeed =  20.438993333333332 m/s
la誤差率 =  0.9423784998476568
lo誤差率 =  0.9496085865337741
finish
Current users number =  5
lat =  24.7940065
lon =  121.0017235
前移 49.523478705361235 公尺
左移 145.69245597013176 公尺
real laspeed =  1650.7826235120413 m/s
real lospeed =  4856.415199004392 m/s
predict laspeed =  20.050673333333336 m/s
predict lospeed =  21.493413333333333 m/s
la誤差率 =  0.9423910551591019
lo誤差率 =  0.9496212772118309
finish
Current users number =  5
lat =  24.7933155
lon =  121.0011635
後移 76.77777777775279 公尺
右移 62.22222222144561 公尺
real laspeed =  2559.2592592584265 m/s
real lospeed =  2074.074074048187 m/s
predict laspeed =  13.132046666666666 m/s
predict lospeed =  18.545960000000004 m/s
la誤差率 =  0.9424055397725141
lo誤差率 =  0.9496327143966626
finish
Current users number =  5
lat =  24.792629
lon =  121.00183
後移 76.27777777743619 公尺
左移 74.05555555490335 公尺
real laspeed =  2542.592592581206 m/s
real lospeed =  2468.5185184967786 m/s
predict laspeed =  17.888493333333333 

Current users number =  5
lat =  24.7934805
lon =  121.001555
前移 12.33333333337959 公尺
左移 61.33333333322197 公尺
real laspeed =  411.111111112653 m/s
real lospeed =  2044.4444444407325 m/s
predict laspeed =  15.783066666666667 m/s
predict lospeed =  20.319773333333334 m/s
la誤差率 =  0.942566622802689
lo誤差率 =  0.9498992690962382
finish
Current users number =  5
lat =  24.792908
lon =  121.0010615
後移 63.6111111111701 公尺
右移 54.83333333226407 公尺
real laspeed =  2120.3703703723368 m/s
real lospeed =  1827.7777777421356 m/s
predict laspeed =  23.63960666666667 m/s
predict lospeed =  16.52136666666667 m/s
la誤差率 =  0.9425793035059947
lo誤差率 =  0.9499105188730358
finish
Current users number =  5
lat =  24.79280151254625
lon =  121.00133806307223
後移 11.83193930545359 公尺
左移 30.729230246606676 公尺
real laspeed =  394.397976848453 m/s
real lospeed =  1024.3076748868893 m/s
predict laspeed =  20.026233333333334 m/s
predict lospeed =  25.788993333333337 m/s
la誤差率 =  0.9425811232765617
lo誤差率 =  0.94991734234